In [11]:
from os import listdir
from os.path import isfile, join
import math
import pandas as pd
import json
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats.mstats import gmean

In [12]:
import numpy as np
import scipy.stats


def confidence_interval(data, confidence=0.99):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return h

In [13]:
root_dir = './results_backup/three_results/'

results=[]
for f in listdir(root_dir):
    path = join(root_dir, f)
    if isfile(path):
        fo = open(path)
        jso = json.loads(fo.read())
        splt = f.split('_')
        tmpvm = splt[3] if len(splt) == 5 else splt[3] + '_' + splt[4]
        if tmpvm.startswith('openj9'):
                vm = 'openj9'
        elif tmpvm.startswith('hotspot'):
                vm = 'hotspot'
        elif tmpvm.startswith('graalvm-ce'):
                vm = 'graalvm CE'
        else:
                vm = 'graalvm EE'
        if '11' in tmpvm:
                vm_version = 11
        elif '12' in tmpvm:
                vm_version = 12
        else:
                vm_version = 8
        timestamp = splt[-1][:-5]
        pattern = splt[1]
        program = splt[2]
        language = splt[0]
        for (idx, time) in enumerate(jso[0]['primaryMetric']['rawData'][0]):
                results.append({
                        'vm': vm,
                        'vm_version': str(vm_version),
                        'vm_full': vm + ' ' + str(vm_version),
                        'pattern': pattern,
                        'program': language[0]+"_"+program,
                        'language': language,
                        'timestamp': timestamp,
                        'time': time,
                        'idx': idx
                })
            
df = pd.DataFrame(results)
for c in ['vm', 'vm_version', 'vm_full', 'pattern', 'program', 'language',]:
    print(c, df[c].unique())
df.describe(include='all')

vm ['openj9' 'hotspot' 'graalvm CE' 'graalvm EE']
vm_version ['12' '8' '11']
vm_full ['openj9 12' 'openj9 8' 'hotspot 8' 'graalvm CE 8' 'hotspot 11'
 'hotspot 12' 'openj9 11' 'graalvm EE 8']
pattern ['visitor' 'interpreter' 'switch' 'revisitor' 'truffle']
program ['f_buffers' 'm_fannkuchredux' 'm_sort' 'm_binarytree' 'b_fibonacci'
 'b_sort' 'm_fibonacci' 'l_fractal']
language ['fsm' 'minijava' 'boa' 'logo']


,vm,vm_version,vm_full,pattern,program,language,timestamp,time,idx
count,39900,39900,39900,39900,39900,39900,39900,39900.000000,39900.000000
unique,4,3,8,5,8,4,798,NaN,NaN
top,openj9,8,graalvm EE 8,interpreter,b_sort,minijava,1565010983779,NaN,NaN
freq,14400,20700,5550,9600,5100,19500,50,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.422510,24.500000
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.019601,14.431051
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.817689,0.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.948882,12.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.470132,24.500000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.753209,37.000000


In [14]:
print('Hotspot VM (Switch)')
hotspot_switch={}
for p in df.program.unique():
    df2 = df.query('vm_version == "8" & vm == "hotspot" & program == "%s" & pattern == "switch" & idx > 10' % p)
    times = df2.time
    mean = times.mean()
    hotspot_switch[p] = { 'mean': mean, 'ci': confidence_interval(times)}
    s = ""
    for c in ['vm', 'vm_version', 'vm_full', 'pattern', 'program', 'language',]:
        s += c + ":" + str(df2[c].unique()) + ","
    print(s)
pd.DataFrame(hotspot_switch)

Hotspot VM (Switch)
vm:['hotspot'],vm_version:['8'],vm_full:['hotspot 8'],pattern:['switch'],program:['f_buffers'],language:['fsm'],
vm:['hotspot'],vm_version:['8'],vm_full:['hotspot 8'],pattern:['switch'],program:['m_fannkuchredux'],language:['minijava'],
vm:['hotspot'],vm_version:['8'],vm_full:['hotspot 8'],pattern:['switch'],program:['m_sort'],language:['minijava'],
vm:['hotspot'],vm_version:['8'],vm_full:['hotspot 8'],pattern:['switch'],program:['m_binarytree'],language:['minijava'],
vm:['hotspot'],vm_version:['8'],vm_full:['hotspot 8'],pattern:['switch'],program:['b_fibonacci'],language:['boa'],
vm:['hotspot'],vm_version:['8'],vm_full:['hotspot 8'],pattern:['switch'],program:['b_sort'],language:['boa'],
vm:['hotspot'],vm_version:['8'],vm_full:['hotspot 8'],pattern:['switch'],program:['m_fibonacci'],language:['minijava'],
vm:['hotspot'],vm_version:['8'],vm_full:['hotspot 8'],pattern:['switch'],program:['l_fractal'],language:['logo'],


,f_buffers,m_fannkuchredux,m_sort,m_binarytree,b_fibonacci,b_sort,m_fibonacci,l_fractal
mean,7.837011,6.666758,16.391664,7.057711,2.044608,1.710339,11.797858,12.440172
ci,0.034134,0.029380,0.017569,0.034380,0.039194,0.013048,0.017202,0.059237


In [15]:
print('Hotspot VM (Interpreter)')
hotspot_interpreter={}
mean_shs=[]
for p in df.program.unique():
    df2 = df.query('vm_version == "8" & vm == "hotspot" & program == "%s" & pattern == "interpreter" & idx > 10' % p)
    times = df2.time
    mean = times.mean()
    shs=hotspot_switch[p]['mean'] / mean
    mean_shs.append(shs)
    hotspot_interpreter[p] = { 'mean': mean, 'ci': confidence_interval(times), 'shs': shs}
    s = ""
    for c in ['vm', 'vm_version', 'vm_full', 'pattern', 'program', 'language',]:
        s += c + ":" + str(df2[c].unique()) + ","
print('mean shs', gmean(mean_shs))
pd.DataFrame(hotspot_interpreter)

Hotspot VM (Interpreter)
mean shs 1.2127438904450678


,f_buffers,m_fannkuchredux,m_sort,m_binarytree,b_fibonacci,b_sort,m_fibonacci,l_fractal
mean,4.044563,5.276449,14.909981,5.858538,1.792009,1.940170,9.181957,11.141054
ci,0.017250,0.020407,0.035728,0.009663,0.063757,0.012230,0.016980,0.054291
shs,1.937666,1.263493,1.099375,1.204688,1.140959,0.881541,1.284896,1.116606


In [16]:
print('Graal VM (Interpreter)')
graal_interpreter={}
for p in df.program.unique():
    df2 = df.query('vm_version == "8" & vm == "graalvm EE" & program == "%s" & pattern == "interpreter"  & idx > 10' % p)
    times = df2.time
    mean = times.mean()
    graal_interpreter[p] = {
        'mean': mean,
        'ci': confidence_interval(times),
        'shs': hotspot_switch[p]['mean'] / mean,
        'shi': hotspot_interpreter[p]['mean'] / mean
    }
    s = ""
    for c in ['vm', 'vm_version', 'vm_full', 'pattern', 'program', 'language',]:
        s += c + ":" + str(df2[c].unique()) + ","
    print(s)
pd.DataFrame(graal_interpreter)

Graal VM (Interpreter)
vm:['graalvm EE'],vm_version:['8'],vm_full:['graalvm EE 8'],pattern:['interpreter'],program:['f_buffers'],language:['fsm'],
vm:['graalvm EE'],vm_version:['8'],vm_full:['graalvm EE 8'],pattern:['interpreter'],program:['m_fannkuchredux'],language:['minijava'],
vm:['graalvm EE'],vm_version:['8'],vm_full:['graalvm EE 8'],pattern:['interpreter'],program:['m_sort'],language:['minijava'],
vm:['graalvm EE'],vm_version:['8'],vm_full:['graalvm EE 8'],pattern:['interpreter'],program:['m_binarytree'],language:['minijava'],
vm:['graalvm EE'],vm_version:['8'],vm_full:['graalvm EE 8'],pattern:['interpreter'],program:['b_fibonacci'],language:['boa'],
vm:['graalvm EE'],vm_version:['8'],vm_full:['graalvm EE 8'],pattern:['interpreter'],program:['b_sort'],language:['boa'],
vm:['graalvm EE'],vm_version:['8'],vm_full:['graalvm EE 8'],pattern:['interpreter'],program:['m_fibonacci'],language:['minijava'],
vm:['graalvm EE'],vm_version:['8'],vm_full:['graalvm EE 8'],pattern:['interpreter'

,f_buffers,m_fannkuchredux,m_sort,m_binarytree,b_fibonacci,b_sort,m_fibonacci,l_fractal
mean,2.556673,3.053266,6.612760,3.497247,1.194350,0.834006,7.644636,4.402155
ci,0.014415,0.023999,0.009267,0.014002,0.018673,0.000973,0.006165,0.044820
shs,3.065315,2.183484,2.478793,2.018076,1.711900,2.050751,1.543286,2.825928
shi,1.581963,1.728133,2.254729,1.675185,1.500405,2.326326,1.201098,2.530818


In [19]:
print('Graal VM (Interpreter)')
truffle_interpreter={}
for p in df.program.unique():
    df2 = df.query('vm_version == "8" & vm == "graalvm EE" & program == "%s" & pattern == "truffle"  & idx > 10' % p)
    times = df2.time
    mean = times.mean()
    truffle_interpreter[p] = {
        'mean': mean,
        'ci': confidence_interval(times),
        'shs': hotspot_switch[p]['mean'] / mean,
        'shi': hotspot_interpreter[p]['mean'] / mean,
        'sgi': graal_interpreter[p]['mean'] / mean,
    }
    s = ""
    for c in ['vm', 'vm_version', 'vm_full', 'pattern', 'program', 'language',]:
        s += c + ":" + str(df2[c].unique()) + ","
    print(s)
pd.DataFrame(truffle_interpreter)

Graal VM (Interpreter)
vm:['graalvm EE'],vm_version:['8'],vm_full:['graalvm EE 8'],pattern:['truffle'],program:['f_buffers'],language:['fsm'],
vm:[],vm_version:[],vm_full:[],pattern:[],program:[],language:[],
vm:[],vm_version:[],vm_full:[],pattern:[],program:[],language:[],
vm:[],vm_version:[],vm_full:[],pattern:[],program:[],language:[],
vm:['graalvm EE'],vm_version:['8'],vm_full:['graalvm EE 8'],pattern:['truffle'],program:['b_fibonacci'],language:['boa'],
vm:['graalvm EE'],vm_version:['8'],vm_full:['graalvm EE 8'],pattern:['truffle'],program:['b_sort'],language:['boa'],
vm:['graalvm EE'],vm_version:['8'],vm_full:['graalvm EE 8'],pattern:['truffle'],program:['m_fibonacci'],language:['minijava'],
vm:['graalvm EE'],vm_version:['8'],vm_full:['graalvm EE 8'],pattern:['truffle'],program:['l_fractal'],language:['logo'],


,f_buffers,m_fannkuchredux,m_sort,m_binarytree,b_fibonacci,b_sort,m_fibonacci,l_fractal
mean,2.331565,NaN,NaN,NaN,1.019734,1.071101,4.629287,4.442771
ci,0.003411,NaN,NaN,NaN,0.017757,0.017548,0.005014,0.046542
shs,3.361267,NaN,NaN,NaN,2.005040,1.596805,2.548526,2.800093
shi,1.734699,NaN,NaN,NaN,1.757329,1.811379,1.983450,2.507681
sgi,1.096548,NaN,NaN,NaN,1.171237,0.778644,1.651364,0.990858
